## 1. Installation & Imports

In [3]:
pip install -q datasets sacrebleu evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00


In [4]:
pip install -U datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [5]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollator, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback

from datasets import load_dataset, DatasetDict

import evaluate

In [7]:
from datasets import load_dataset
import numpy as np
import torch

## 2. Dataset Loading & Preparation

- Merging multiple dataset versions
- Removing unnecessary columns
- Splitting data into train/val/test sets

In [9]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

v1 = load_dataset("cnn_dailymail", "1.0.0", split="train[:20000]")
v2 = load_dataset("cnn_dailymail", "2.0.0", split="train[:20000]")
v3 = load_dataset("cnn_dailymail", "3.0.0", split="train[:20000]")

data = concatenate_datasets([v1, v2, v3])

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [10]:
data = data.remove_columns("id")

In [11]:
data

Dataset({
    features: ['article', 'highlights'],
    num_rows: 60000
})

In [12]:
d1 = data.train_test_split(test_size=0.2)

In [13]:
d1

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 48000
    })
    test: Dataset({
        features: ['article', 'highlights'],
        num_rows: 12000
    })
})

In [14]:
d1['train']

Dataset({
    features: ['article', 'highlights'],
    num_rows: 48000
})

In [15]:
d2 = d1['train'].train_test_split(test_size=0.2)

In [16]:
d2

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 38400
    })
    test: Dataset({
        features: ['article', 'highlights'],
        num_rows: 9600
    })
})

### Splitting Data into Train/Val/Test

In [17]:
final_data = DatasetDict({'train' : d2['train'], 'validation' : d2['test'], 'test' : d1['test']})

In [18]:
final_data

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 38400
    })
    validation: Dataset({
        features: ['article', 'highlights'],
        num_rows: 9600
    })
    test: Dataset({
        features: ['article', 'highlights'],
        num_rows: 12000
    })
})

## 3. Model & Tokenizer Setup
- Loading the T5 model and tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration, T5TokenizerFast

model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
tokenizer = T5TokenizerFast.from_pretrained("google-t5/t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

## 4. Preprocessing Preprocessing Function for Tokenization

In [19]:
max_input_length = 512
max_target_length = 128

def preprocess(example):
    inputs = ["summarize: " + article for article in example["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="longest")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["highlights"], max_length=max_target_length, truncation=True, padding="longest")

    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels_seq]
        for labels_seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = final_data.map(preprocess, batched=True, remove_columns=["article", "highlights"])

Map:   0%|          | 0/38400 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [20]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 38400
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9600
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 12000
    })
})

## 5. Evaluation Metrics
- ROUGE score setup

In [ ]:
# Metric
import evaluate
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in labels (used as ignore index) with pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Strip leading/trailing whitespaces
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Return the scalar float values directly
    return {
        "rouge1": result["rouge1"] * 100,
        "rouge2": result["rouge2"] * 100,
        "rougeL": result["rougeL"] * 100,
        "rougeLsum": result["rougeLsum"] * 100,
    }

## 6. Training the Model
- Training arguments
- Trainer initialization
- Model training

In [ ]:
# Training arguments
ta = Seq2SeqTrainingArguments(
    output_dir="./T5-small/results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    learning_rate=5e-5,
    num_train_epochs=5,
    predict_with_generate=True,
    report_to=["wandb"],
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    run_name="T5-small-summarization",
    fp16=True
)


In [ ]:
# Trainer
tr = Seq2SeqTrainer(
    model=model,
    args=ta,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

<ipython-input-34-2558c9c8406c>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  tr = Seq2SeqTrainer(


In [ ]:
# Train
model = tr.train()

## Load Fine-Tuned Model from Latest Checkpoint and Inference on 10 Sample Inputs

In [ ]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast

# Load your fine-tuned model
model_path = "/content/T5-small/results/checkpoint-6400"  # path to fine-tuned model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5TokenizerFast.from_pretrained(model_path)

# 10 sample input texts for summarization
sample_texts = [
    "summarize: Climate change is accelerating faster than previously predicted. Global temperatures are rising, leading to extreme weather events like hurricanes, droughts, and floods. Scientists urge immediate reduction of greenhouse gas emissions to prevent irreversible damage.",

    "summarize: The Indian economy grew at 6.1% in the last quarter, driven by strong performance in manufacturing and services. The Reserve Bank of India may consider adjusting interest rates in response to inflation trends and global economic pressures.",

    "summarize: In the final match of the FIFA World Cup, Argentina beat France in a thrilling penalty shootout. Messi scored twice, securing his legacy as one of the greatest footballers of all time. The match was widely regarded as one of the most exciting in history.",

    "summarize: Apple's new iPhone 15 introduces a periscope zoom camera, titanium frame, and USB-C port. It also features improvements in battery life and AI-powered photography. Reviews have been largely positive, praising performance and design.",

    "summarize: A new study shows that a Mediterranean diet, rich in vegetables, olive oil, and fish, can significantly reduce the risk of heart disease. Participants also reported improved mental clarity and energy levels.",

    "summarize: The Artemis I mission successfully completed its journey around the moon and returned safely to Earth. NASA plans to send astronauts to the moon in the next phase, establishing a long-term human presence as part of the Artemis program.",

    "summarize: The COVID-19 pandemic exposed vulnerabilities in global healthcare systems. Many countries faced shortages of medical supplies, staff, and ICU beds. Lessons learned have led to new policies aimed at future pandemic preparedness.",

    "summarize: Elon Musk's acquisition of Twitter sparked both praise and controversy. He implemented mass layoffs, introduced paid verification, and promised more transparency in algorithms. Users and advertisers remain divided over the platform’s direction.",

    "summarize: Researchers have developed a breakthrough cancer treatment using mRNA technology. Early trials show promising results in targeting tumors with minimal side effects, potentially revolutionizing oncology in the next decade.",

    "summarize: A massive volcanic eruption in Iceland disrupted air travel across Europe. Ash clouds spread rapidly, grounding flights and affecting millions of travelers. Emergency services were deployed to monitor and manage the situation."
]

# Generate summaries
for i, text in enumerate(sample_texts):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=150,
        min_length=30,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(f"\nText {i+1} Summary:\n{summary}")



Text 1 Summary:
Climate change is accelerating faster than previously predicted. Global temperatures are rising, leading to extreme weather events like hurricanes, droughts, and floods.

Text 2 Summary:
The Indian economy grew at 6.1% in the last quarter, driven by strong performance in manufacturing and services. The Reserve Bank of India may consider adjusting interest rates.

Text 3 Summary:
Argentina beat France in a thrilling penalty shootout in the final match of the FIFA World Cup. Messi scored twice, securing his legacy as one of the greatest players of all time.

Text 4 Summary:
Apple's new iPhone 15 features a periscope zoom camera, titanium frame, and USB-C port. Reviews have been largely positive, praising performance and design.

Text 5 Summary:
Mediterranean diet, rich in vegetables, olive oil, and fish, can significantly reduce heart disease risk. Participants also reported improved mental clarity and energy levels.

Text 6 Summary:
The Artemis I mission successfully co

In [ ]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast

model = T5ForConditionalGeneration.from_pretrained("/content/T5-small/results/checkpoint-6400")
tokenizer = T5TokenizerFast.from_pretrained("/content/T5-small/results/checkpoint-6400")

## 7. Deploying model on HuggingFace

In [ ]:
!pip install -q huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("NeonSamurai/summarization_t5_small_v2")
tokenizer.push_to_hub("NeonSamurai/summarization_t5_small_v2")

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NeonSamurai/summarization_t5_small_v2/commit/4d4cd6e6eff1e34130f33f9036ed7ba03e1d0a7e', commit_message='Upload tokenizer', commit_description='', oid='4d4cd6e6eff1e34130f33f9036ed7ba03e1d0a7e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NeonSamurai/summarization_t5_small_v2', endpoint='https://huggingface.co', repo_type='model', repo_id='NeonSamurai/summarization_t5_small_v2'), pr_revision=None, pr_num=None)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="NeonSamurai/summarization_t5_small_v2")

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
pipe("summarize: Climate change is accelerating faster than previously predicted. Global temperatures are rising, leading to extreme weather events like hurricanes, droughts, and floods. Scientists urge immediate reduction of greenhouse gas emissions to prevent irreversible damage.")

[{'generated_text': 'Global temperatures are rising, leading to extreme weather events like hurricanes, droughts, and floods . Scientists urge immediate reduction of greenhouse gas emissions to prevent irreversible damage .'}]

In [ ]:
pipe("""The Trump administration has ordered US embassies worldwide to immediately stop scheduling visa interviews for foreign students as it prepares to implement comprehensive social media screening for all international applicants.

A Tuesday state department cable instructs consular sections to pause adding “any additional student or exchange visitor (F, M, and J) visa appointment capacity until further guidance is issued” within days.

The directive, first reported by Politico and now confirmed by the Guardian, could severely delay visa processing and hurt universities – many of which Donald Trump accuses of having far-left ideologies – that rely heavily on foreign students for revenue.

“The department is conducting a review of existing operations and processes for screening and vetting of student and exchange visitor visa applicants,” the cable reads. Officials plan to issue guidance on “expanded social media vetting for all such applicants”.

""")

[{'generated_text': 'applicants could severely delay visa processing and hurt universities . The Trump administration has ordered US embassies worldwide to stop scheduling visa interviews for foreign students . The Trump administration has ordered US embassies worldwide to immediately stop scheduling visa interviews for foreign students . The Trump administration has ordered US embassies worldwide to immediately stop scheduling visa interviews for foreign students .'}]

## 8. Extractive vs Abstractive Summarization
- Using spaCy as an extractive method
- Comparing with T5 summaries


In [ ]:
import spacy
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from collections import Counter
import string

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load T5 model
model_path = "/content/T5-small/results/checkpoint-6400"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5TokenizerFast.from_pretrained(model_path)

# Input texts
texts = [
    "summarize: The global climate crisis has reached a critical point, with temperatures rising faster than expected. Scientists have observed a dramatic increase in ice melt in both the Arctic and Antarctic regions. Sea levels are projected to rise significantly by the end of the century, threatening coastal cities worldwide. Forest fires, heatwaves, and hurricanes are becoming more frequent and intense, indicating the need for urgent action. International agreements such as the Paris Accord have called for reducing emissions, but many countries are falling short of their targets. Experts emphasize the need for systemic change in energy production, transportation, and agriculture to curb greenhouse gases effectively."
]

def extractive_summary_spacy(text, top_n=3):
    doc = nlp(text)
    words = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop]
    freq = Counter(words)

    # Score sentences based on word frequencies
    sent_scores = {}
    for sent in doc.sents:
        score = sum(freq.get(token.text.lower(), 0) for token in sent if token.is_alpha)
        sent_scores[sent] = score

    # Get top n sentences
    top_sents = sorted(sent_scores, key=sent_scores.get, reverse=True)[:top_n]
    return ' '.join([sent.text for sent in top_sents])

# Compare summaries
for i, text in enumerate(texts):
    # Abstractive
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    abstractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Extractive
    original_text = text.replace("summarize: ", "")
    extractive_summary = extractive_summary_spacy(original_text, top_n=3)

    # Print
    print(f"\n--- Text {i+1} Comparison ---")
    print(f"Original Length: {len(original_text)} chars")

    print(f"\nAbstractive Summary:\n{abstractive_summary}")
    print(f"Abstractive Length: {len(abstractive_summary)} chars")

    print(f"\nExtractive Summary:\n{extractive_summary}")
    print(f"Extractive Length: {len(extractive_summary)} chars")



--- Text 1 Comparison ---
Original Length: 712 chars

Abstractive Summary:
The global climate crisis has reached a critical point, with temperatures rising faster than expected. Scientists have observed a dramatic increase in ice melt in both the Arctic and Antarctic regions. Sea levels are projected to rise significantly by the end of the century.
Abstractive Length: 275 chars

Extractive Summary:
Experts emphasize the need for systemic change in energy production, transportation, and agriculture to curb greenhouse gases effectively. Sea levels are projected to rise significantly by the end of the century, threatening coastal cities worldwide. Forest fires, heatwaves, and hurricanes are becoming more frequent and intense, indicating the need for urgent action.
Extractive Length: 369 chars


## 9. Evaluation & Final Metrics
- Final predictions
- ROUGE metrics
- Summary DataFrame

In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("NeonSamurai/summarization_t5_small_v2")
model = AutoModelForSeq2SeqLM.from_pretrained("NeonSamurai/summarization_t5_small_v2")

In [22]:
from transformers import DataCollatorForSeq2Seq
from transformers import EarlyStoppingCallback
import evaluate
import numpy as np

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Metric
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in labels (used as ignore index) with pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Strip leading/trailing whitespaces
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Return the scalar float values directly
    return {
        "rouge1": result["rouge1"] * 100,
        "rouge2": result["rouge2"] * 100,
        "rougeL": result["rougeL"] * 100,
        "rougeLsum": result["rougeLsum"] * 100,
    }


# Training arguments
ta = Seq2SeqTrainingArguments(
    output_dir="./T5-small/results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    learning_rate=5e-5,
    num_train_epochs=5,
    predict_with_generate=True,
    report_to=["wandb"],
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    run_name="T5-small-summarization",
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=ta, # Pass the training arguments
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

<ipython-input-22-1868421404>:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [23]:
# Predict on test set
test_results = trainer.predict(tokenized_dataset["test"])

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: suhaib-aamer (suhaib-aamer-innomatics-research-labs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [24]:
# Decode predictions
predictions = tokenizer.batch_decode(test_results.predictions, skip_special_tokens=True)

In [25]:
import numpy as np

# Replace -100 (ignored label tokens) with pad_token_id before decoding
labels = np.where(test_results.label_ids != -100, test_results.label_ids, tokenizer.pad_token_id)
references = tokenizer.batch_decode(labels, skip_special_tokens=True)

In [26]:
import pandas as pd
pd.DataFrame({"generated_summary": predictions})

,generated_summary
0,NEW: United Airlines says it will begin chargi...
1,Fighting between Somalia's transitional govern...
2,Ricky Gervais plays dentist named Bertram Pinc...
3,NEW: O'Donnell's publicist says she's inspired...
4,Former President Clinton says Republicans won'...
...,...
11995,"Libyan currency dealers used ""parallar markets..."
11996,NBA player Donté Greene was a lifesaver on Mem...
11997,NEW: DOT says it is balancing desire of consum...
11998,North Korean state media announces Kim Jong Un...


In [28]:
rouge = evaluate.load("rouge")

# Strip whitespace
preds = [pred.strip() for pred in predictions]
refs = [ref.strip() for ref in references]

# Compute ROUGE
results = rouge.compute(predictions=preds, references=refs, use_stemmer=True)

# Display
for key, value in results.items():
    print(f"{key}: {value * 100:.2f}")

rouge1: 24.63
rouge2: 10.54
rougeL: 20.01
rougeLsum: 20.01
